In [2]:
import os
from collections import namedtuple
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use("agg")
%matplotlib inline

run_local = False

region_name = "Demo"
dir_name = region_name
if not os.path.exists(dir_name):
    os.mkdir(dir_name)
root_dir = lambda x: os.path.join(dir_name, x)
Artifact2 = namedtuple("Artifact", ["path"])

In [4]:
from kfp import dsl

@dsl.component
def square(x: float) -> float:
    return x ** 2

In [9]:
square.component_spec

ComponentSpec(name='square', implementation=Implementation(container=ContainerSpecImplementation(image='python:3.7', command=['sh', '-c', '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quiet     --no-warn-script-location \'kfp==2.0.1\' && "$0" "$@"\n', 'sh', '-ec', 'program_path=$(mktemp -d)\nprintf "%s" "$0" > "$program_path/ephemeral_component.py"\npython3 -m kfp.components.executor_main                         --component_module_path                         "$program_path/ephemeral_component.py"                         "$@"\n', '\nimport kfp\nfrom kfp import dsl\nfrom kfp.dsl import *\nfrom typing import *\n\ndef square(x: float) -> float:\n    return x ** 2\n\n'], args=['--executor_input', <kfp.components.placeholders.ExecutorInputPlaceholder object at 0x11673f490>, '--function_to_execute', 'square'], env=None, resources=None), importer=N

In [2]:
from kfp import client
from kfp import dsl
from kfp.dsl import component, Input, Output, Artifact, OutputPath, InputPath

In [3]:
@dsl.component(packages_to_install=["obspy", "numpy", "minio", "matplotlib"])
def set_config(
    region_name: str,
    index_json: Output[Artifact],
    config_json: Output[Artifact],
    datetime_json: Output[Artifact],
    num_parallel: int = 1,
    bucket_name: str = "catalogs",
    s3_url: str = "minio-service:9000",
    secure: bool = True,
):

    import datetime
    import json
    import os
    import pickle
    import numpy as np
    import obspy

    degree2km = np.pi * 6371 / 180

    if region_name == "Demo":
        center = (-117.504, 35.705)
        horizontal_degree = 1.0
        vertical_degree = 1.0
        starttime = obspy.UTCDateTime("2019-07-04T17")
        endtime = obspy.UTCDateTime("2019-07-04T19")
        client = "SCEDC"
        network_list = ["CI"]
        channel_list = "HH*,BH*,EH*,HN*"


    ####### save config ########
    config = {}
    config["region"] = region_name
    config["center"] = center
    config["xlim_degree"] = [
        center[0] - horizontal_degree / 2,
        center[0] + horizontal_degree / 2,
    ]
    config["ylim_degree"] = [
        center[1] - vertical_degree / 2,
        center[1] + vertical_degree / 2,
    ]
    config["min_longitude"] = center[0] - horizontal_degree / 2
    config["max_longitude"] = center[0] + horizontal_degree / 2
    config["min_latitude"] = center[1] - vertical_degree / 2
    config["max_latitude"] = center[1] + vertical_degree / 2
    config["degree2km"] = degree2km
    config["starttime"] = starttime.datetime.isoformat(timespec="milliseconds")
    config["endtime"] = endtime.datetime.isoformat(timespec="milliseconds")
    config["networks"] = network_list
    config["channels"] = channel_list
    config["client"] = client

    ## PhaseNet
    config["phasenet"] = {}
    ## GaMMA
    config["gamma"] = {}
    ## HypoDD
    config["hypodd"] = {"MAXEVENT": 1e4}

    with open(config_json.path, "w") as fp:
        json.dump(config, fp, indent=2)

    print(json.dumps(config, indent=4))

    ####### set paraell for cloud ########
    ## split data by hours
    # one_day = datetime.timedelta(days=1)
    one_hour = datetime.timedelta(hours=1)
    starttimes = []
    tmp_start = starttime
    while tmp_start < endtime:
        starttimes.append(tmp_start.datetime.isoformat(timespec="milliseconds"))
        tmp_start += one_hour
    with open(datetime_json.path, "w") as fp:
        json.dump(
            {"starttimes": starttimes, "interval": one_hour.total_seconds()},
            fp,
            indent=2,
        )

    ## split stattimes into N parallel process
    if num_parallel == 0:
        num_parallel = min(60, int((len(starttimes) - 1) // 6 + 1))
        # num_parallel = min(30, int((len(starttimes)-1)//6+1))
        # num_parallel = min(24, len(starttimes))
    idx = [x.tolist() for x in np.array_split(np.arange(len(starttimes)), num_parallel)]

    with open(index_json.path, "w") as fp:
        json.dump(idx, fp, indent=2)

    ## upload to s3 bucket
    try:
        from minio import Minio

        minioClient = Minio(s3_url, access_key="minio", secret_key="minio123", secure=secure)
        if not minioClient.bucket_exists(bucket_name):
            minioClient.make_bucket(bucket_name)

        minioClient.fput_object(
            bucket_name,
            f"{config['region']}/config.json",
            config_json,
        )

        minioClient.fput_object(
            bucket_name,
            f"{config['region']}/datetime.json",
            datetime_json,
        )

        minioClient.fput_object(
            bucket_name,
            f"{config['region']}/index.json",
            index_json,
        )

    except Exception as err:
        print(f"ERROR: can not access minio service! \n{err}")
        pass

In [4]:
if run_local:
    set_config.python_func(region_name="Demo", index_json=Artifact2(root_dir("index.json")), config_json=Artifact2(root_dir("config.json")), datetime_json=Artifact2(root_dir("datetime.json")), num_parallel=1, bucket_name="catalogs", s3_url="minio-service:9000", secure=True)

In [5]:
@dsl.component(packages_to_install=["obspy", "numpy", "minio", "pandas", "matplotlib"])
def download_events(
    config_json: Input[Artifact], 
    standard_catalog_csv: Output[Artifact],
    catalog_location_png: Output[Artifact],
    catalog_magnitude_png: Output[Artifact],
    bucket_name: str = "catalogs",
    s3_url: str = "minio-service:9000",
    secure: bool = True,
):

    import json
    import os
    import pickle
    from collections import defaultdict

    import matplotlib
    import matplotlib.pyplot as plt
    import obspy
    import pandas as pd
    from obspy.clients.fdsn import Client

    # matplotlib.use("agg")
    # %matplotlib inline

    with open(config_json.path, "r") as fp:
        config = json.load(fp)

    ####### IRIS catalog ########
    try:
        events = Client(config["client"]).get_events(
            starttime=config["starttime"],
            endtime=config["endtime"],
            minlongitude=config["xlim_degree"][0],
            maxlongitude=config["xlim_degree"][1],
            minlatitude=config["ylim_degree"][0],
            maxlatitude=config["ylim_degree"][1],
            # filename='events.xml',
        )
    except:
        events = Client("USGS").get_events(
            starttime=config["starttime"],
            endtime=config["endtime"],
            minlongitude=config["xlim_degree"][0],
            maxlongitude=config["xlim_degree"][1],
            minlatitude=config["ylim_degree"][0],
            maxlatitude=config["ylim_degree"][1],
            # filename='events.xml',
        )

    print(f"Number of events: {len(events)}")

    ####### Save catalog ########
    catalog = defaultdict(list)
    for event in events:
        if len(event.magnitudes) > 0:
            catalog["time"].append(event.origins[0].time.datetime)
            catalog["magnitude"].append(event.magnitudes[0].mag)
            catalog["longitude"].append(event.origins[0].longitude)
            catalog["latitude"].append(event.origins[0].latitude)
            catalog["depth(m)"].append(event.origins[0].depth)
    catalog = pd.DataFrame.from_dict(catalog).sort_values(["time"])
    catalog.to_csv(
        standard_catalog_csv.path,
        index=False,
        float_format="%.3f",
        date_format="%Y-%m-%dT%H:%M:%S.%f",
        columns=["time", "magnitude", "longitude", "latitude", "depth(m)"],
    )

    ## upload to s3 bucket
    try:
        from minio import Minio

        minioClient = Minio(s3_url, access_key="minio", secret_key="minio123", secure=secure)
        if not minioClient.bucket_exists(bucket_name):
            minioClient.make_bucket(bucket_name)

        minioClient.fput_object(
            bucket_name,
            f"{config['region']}/standard_catalog.csv",
            standard_catalog_csv,
        )

    except Exception as err:
        print(f"ERROR: can not access minio service! \n{err}")
        pass

    ####### Plot catalog ########
    plt.figure()
    plt.plot(catalog["longitude"], catalog["latitude"], ".", markersize=1)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.axis("scaled")
    plt.xlim(config["xlim_degree"])
    plt.ylim(config["ylim_degree"])
    plt.savefig(catalog_location_png.path, format="png", dpi=300)

    plt.figure()
    plt.plot_date(catalog["time"], catalog["magnitude"], ".", markersize=1)
    plt.gcf().autofmt_xdate()
    plt.ylabel("Magnitude")
    plt.title(f"Number of events: {len(events)}")
    plt.savefig(catalog_magnitude_png.path, format="png", dpi=300)

In [6]:
@dsl.pipeline(name='quakeflow-pipeline')
def quakeflow_pipeline(
    data_path: str = "/tmp/",
    num_parallel: int = 0,
    bucket_catalog: str = "catalogs",
    s3_url: str = "minio-service:9000",
    secure: bool = False,
):

    config = set_config(
        region_name=region_name, num_parallel=num_parallel, bucket_name=f"catalogs", s3_url=s3_url, secure=secure
    )

    events = download_events(config_json = config.outputs["config_json"], bucket_name=f"catalogs", s3_url=s3_url, secure=secure)
    

    


In [9]:
endpoint = 'http://test.quakeflow.com:8080'
kfp_client = client.Client(host=endpoint)
run = kfp_client.create_run_from_pipeline_func(
    quakeflow_pipeline,
    # arguments={},
)
url = f'{endpoint}/#/runs/details/{run.run_id}'
print(url)

/Users/weiqiang/.local/miniconda3/lib/python3.9/site-packages/kfp/client/client.py:158: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


http://test.quakeflow.com:8080/#/runs/details/5b0a8d58-a561-4e31-b962-9738309b523b
